In [1]:
import numpy as np
import pandas as pd
import timeit

In [2]:
train_orig =  pd.read_csv('data/train.csv', header=0)
test_orig =  pd.read_csv('data/test.csv', header=0)


In [3]:
df1 = train_orig[['question1']].copy()
df2 = train_orig[['question2']].copy()
df1_test = test_orig[['question1']].copy()
df2_test = test_orig[['question2']].copy()


In [5]:
df2.rename(columns = {'question2':'question1'},inplace=True)
df2_test.rename(columns = {'question2':'question1'},inplace=True)


In [7]:
train_questions = df1.append(df2)
train_questions = train_questions.append(df1_test)
train_questions = train_questions.append(df2_test)
#train_questions.drop_duplicates(subset = ['qid1'],inplace=True)
train_questions.drop_duplicates(subset = ['question1'],inplace=True)


In [8]:
train_questions.reset_index(inplace=True,drop=True) #combining all questions, no duplicates 

In [10]:
questions_dict = pd.Series(train_questions.index.values,index=train_questions.question1.values).to_dict()

In [16]:
train_cp = train_orig.copy()
test_cp = test_orig.copy()
train_cp.drop(['qid1','qid2'],axis=1,inplace=True)

test_cp['is_duplicate'] = -1
test_cp.rename(columns={'test_id':'id'},inplace=True)
comb = pd.concat([train_cp,test_cp])

In [18]:
len(comb), len(test_cp)

(2750086, 2345796)

In [19]:
comb['q1_hash'] = comb['question1'].map(questions_dict)
comb['q2_hash'] = comb['question2'].map(questions_dict)


In [21]:
# comb['q1_hash'] 

In [22]:
q1_vc = comb.q1_hash.value_counts().to_dict()
q2_vc = comb.q2_hash.value_counts().to_dict()


In [25]:
def try_apply_dict(x,dict_to_apply):
    try:
        return dict_to_apply[x]
    except KeyError:
        return 0

In [35]:
comb['q1_hash'][1999:2001]

1999    1991
2000    1992
Name: q1_hash, dtype: int64

In [28]:
#map to frequency space
comb['q1_freq'] = comb['q1_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))
comb['q2_freq'] = comb['q2_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))


In [38]:
train_comb = comb[comb['is_duplicate'] >= 0][['id','q1_hash','q2_hash','q1_freq','q2_freq','is_duplicate']]
test_comb = comb[comb['is_duplicate'] < 0][['id','q1_hash','q2_hash','q1_freq','q2_freq']]

In [40]:
corr_mat = train_comb.corr()
corr_mat

,id,q1_hash,q2_hash,q1_freq,q2_freq,is_duplicate
id,1.000000,0.692730,0.286969,-0.001608,-0.000777,-0.008784
q1_hash,0.692730,1.000000,0.492993,-0.341777,-0.202545,-0.206498
q2_hash,0.286969,0.492993,1.000000,-0.392605,-0.466434,-0.349626
q1_freq,-0.001608,-0.341777,-0.392605,1.000000,0.494315,0.296621
q2_freq,-0.000777,-0.202545,-0.466434,0.494315,1.000000,0.198609
is_duplicate,-0.008784,-0.206498,-0.349626,0.296621,0.198609,1.000000
